In [1]:
import awswrangler as awr
import pandas as pd

file_path = r"C:\Users\raphael.almeida\Documents\Processos\historico_placas\sql\all_boards_sets.sql"
with open (file_path, 'r') as r:
    query = r.read()

df_sets = awr.athena.read_sql_query(query, database='silver')

CONFERÊNCIAS

In [2]:
df_sets['status_conjunto'].unique()
# se ATIVO NOVO RENOVAÇÃO é placa vigente, porque tem placas que nunca mudam para ATIVO, ficam como estão
# ['CANCELADO', 'CANCELADA', 'FINALIZADO', 'FINALIZADA', 'NAO RENOVADO']

<StringArray>
[             'CANCELADO',             'FINALIZADO',                  'ATIVO',
              'RENOVAÇÃO',            'DESISTÊNCIA',                   'NOVO',
  'AGUARDANDO ASSINATURA', 'DEVOLVIDO PARA UNIDADE', 'AGENDADO PARA ATIVAÇÃO',
            'SUBSTITUÍDO',      'AGUARDANDO ATIVAR',           'NAO RENOVADO',
                 'NEGADO',                     <NA>]
Length: 14, dtype: string

In [3]:
df_sets1 = df_sets[df_sets['placa']=='ZZZ9999']
df_sets1.sort_values('data_registro', ascending=False)

,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao
46155,ZZZ9999,528LKACKDXPPD244,186298,2149,2833,2024-09-03,None,2024-09-03,SUBSTITUÍDO,Viavante,2025-04-22


criando cancelamentos e ativações

In [11]:
df_ativacoes = df_sets[df_sets['data_cancelamento_conjunto'].isnull()]

df_ativacoes = df_ativacoes[df_ativacoes['status_conjunto'].isin(['ATIVO', 'NOVO', 'RENOVAÇÃO'])] #averiguar o novo e renovação

df_ativacoes = df_ativacoes.drop_duplicates()

df_ativacoes['data_status'] = df_ativacoes['data_ativacao_conjunto']

df_ativacoes['vigencia'] = 'ATIVO'

In [12]:
df_cancelamentos = df_sets[df_sets['data_cancelamento_conjunto'].notna()]

df_cancelamentos = df_cancelamentos.drop_duplicates()

df_cancelamentos['data_status'] = df_cancelamentos['data_cancelamento_conjunto']

df_cancelamentos['vigencia'] = 'CANCELADO'

juntando as duas e filtrando ontem

In [25]:
df_aggregate = pd.concat([df_ativacoes, df_cancelamentos])


today = pd.Timestamp.today()
today_date = today.date()
yesterday = today - pd.Timedelta(days=1)
yesterday_date= yesterday.date()

df_aggregate['data_status'] = pd.to_datetime(df_aggregate['data_status'])
df_aggregate_yesterday = df_aggregate[df_aggregate['data_status'].dt.date==yesterday_date]

#df_aggregate.sort_values('data_ativacao_conjunto', ascending=False)
df_aggregate_yesterday.sort_values('placa', ascending=False)

df_aggregate_yesterday['status'] = pd.NA
df_aggregate_yesterday['migration_from'] = pd.NA

df_aggregate_yesterday

C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_10900\585305305.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aggregate_yesterday['status'] = pd.NA
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_10900\585305305.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aggregate_yesterday['migration_from'] = pd.NA


,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao,data_status,vigencia,status,migration_from
908,AZD1400,9BSR6X200D3820864,25199,4731,14379,2025-05-07,None,2025-05-15,ATIVO,Stcoop,2025-05-16,2025-05-15,ATIVO,<NA>,<NA>
3795,EJX5E33,9BVAS50D5AE757022,8487,20610,14441,2025-05-14,None,2025-05-15,ATIVO,Stcoop,2025-05-15,2025-05-15,ATIVO,<NA>,<NA>
7253,SCZ4A16,91VG1354RRC213838,18955,22249,14426,2025-05-13,None,2025-05-15,ATIVO,Stcoop,2025-05-15,2025-05-15,ATIVO,<NA>,<NA>
7254,TBB1I10,3GNAX9EB0SL175330,27921,22250,14428,2025-05-14,None,2025-05-15,ATIVO,Stcoop,2025-05-15,2025-05-15,ATIVO,<NA>,<NA>
7439,NTX3096,9BSG4X200B3675274,25098,21028,14347,2025-05-05,None,2025-05-15,ATIVO,Stcoop,2025-05-16,2025-05-15,ATIVO,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54220,MLF3F69,96AB1553EFJ001696,436,50,584,2024-05-14,2025-05-15,2024-05-14,FINALIZADO,Viavante,2025-05-15,2025-05-15,CANCELADO,<NA>,<NA>
56328,AYS5I26,9BSR6X200E3861757,92107,383,412,2024-04-29,2025-05-15,2024-04-29,FINALIZADO,Viavante,2025-04-29,2025-05-15,CANCELADO,<NA>,<NA>
56853,MCI7G11,9EP07082021001082,8129,4078,10072,2025-02-13,2025-05-15,2025-02-14,CANCELADO,Viavante,2025-05-15,2025-05-15,CANCELADO,<NA>,<NA>
58616,KEQ3I47,9ADG071222M177707,7311,2926,10046,2025-02-13,2025-05-15,2025-02-13,CANCELADO,Viavante,2025-05-15,2025-05-15,CANCELADO,<NA>,<NA>


gerando a base geral de ativos

In [34]:
import pandas as pd
import awswrangler as awr
import openpyxl

file_path = r"C:\Users\raphael.almeida\Documents\Processos\historico_placas\sql\all_boards_sets.sql"

with open (file_path, 'r') as file:
    query = file.read()

df_conf = awr.athena.read_sql_query(query, database='silver')


tratando a base geral 

tratando a data

In [35]:


df_conf['data_ativacao_conjunto'] = pd.to_datetime(df_conf['data_ativacao_conjunto'])
df_conf['data_cancelamento_conjunto'] = pd.to_datetime(df_conf['data_cancelamento_conjunto'])

df_conf = df_conf[
    (df_conf['data_ativacao_conjunto'].dt.date!=yesterday_date) &
    (df_conf['data_ativacao_conjunto'].dt.date!=today_date) &
    (df_conf['data_cancelamento_conjunto'].dt.date!=yesterday_date) &
    (df_conf['data_cancelamento_conjunto'].dt.date!=today_date)
    ]



colocando data cancel e ativ em uma só coluna

In [36]:
df_ativacoes_geral = df_conf[df_conf['data_cancelamento_conjunto'].isnull()]

df_ativacoes_geral = df_ativacoes_geral[df_ativacoes_geral['status_conjunto'].isin(['ATIVO', 'NOVO', 'RENOVAÇÃO'])] #averiguar o novo e renovação

df_ativacoes_geral['data_status'] = df_ativacoes_geral['data_ativacao_conjunto']

df_ativacoes_geral['vigencia'] = 'ATIVO'

######################################################################

df_cancelamentos_geral = df_conf[df_conf['data_cancelamento_conjunto'].notna()]

df_cancelamentos_geral['data_status'] = df_cancelamentos_geral['data_cancelamento_conjunto']

df_cancelamentos_geral['vigencia'] = 'CANCELADO'

######################## CONCATENANDO

df_conf = pd.concat([df_ativacoes_geral, df_cancelamentos_geral])

C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_10900\1336788432.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cancelamentos_geral['data_status'] = df_cancelamentos_geral['data_cancelamento_conjunto']
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_10900\1336788432.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cancelamentos_geral['vigencia'] = 'CANCELADO'


In [37]:
df_conf

,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao,data_status,vigencia
0,ISR3D96,9BVAS02A4BE781139,18928,12012,13495,2025-02-05,NaT,2025-02-14,ATIVO,Stcoop,2025-02-15,2025-02-14,ATIVO
8,DBM2G08,9BWKR82T54R431588,14492,12122,4399,2024-01-04,NaT,2024-01-08,RENOVAÇÃO,Stcoop,2025-01-08,2024-01-08,ATIVO
11,BBW4C92,94BA1253JJV055276,3696,12126,2147,2023-10-25,NaT,2023-10-26,RENOVAÇÃO,Stcoop,2024-10-26,2023-10-26,ATIVO
12,APN9J97,9BSR6X200D3815381,6016,12126,2147,2023-10-25,NaT,2023-10-26,RENOVAÇÃO,Stcoop,2024-10-26,2023-10-26,ATIVO
13,AUN4F40,9BVAS02C5BE779708,13481,12133,12865,2024-12-19,NaT,2024-12-23,ATIVO,Stcoop,2024-12-23,2024-12-23,ATIVO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
242585,CPJ4F06,9AU18123011035366,17600,6551,10947,2025-02-26,2025-03-27,2025-02-26,CANCELADO,Viavante,2025-03-27,2025-03-27,CANCELADO
242644,AON8F90,9BFYCEHV67BB87646,16032856,6590,10461,2025-02-19,2025-03-12,2025-02-19,CANCELADO,Viavante,2025-03-12,2025-03-12,CANCELADO
242645,QVI7E24,97T0AN452LC008310,17408,6593,10465,2025-02-19,2025-04-25,2025-02-21,CANCELADO,Viavante,2025-04-25,2025-04-25,CANCELADO
242662,AWJ4F00,9BSR4X2A073615029,15986347,6627,10524,2025-02-20,2025-04-28,2025-02-21,CANCELADO,Viavante,2025-04-28,2025-04-28,CANCELADO


criando as condições do status baeado no histórico montado

In [38]:
# Agrupar dados de conferência por chassi e benefício
df_conf_grouped = df_conf.groupby(['chassi']).agg({
    'empresa': list,
    'vigencia': list,
    'data_status': list
}).reset_index()

# Criar um dicionário para lookup rápido
conf_dict = df_conf_grouped.set_index(['chassi']).to_dict('index')

# Processar cada chassi/benefício único (validação)
for chassi in df_aggregate_yesterday['chassi'].drop_duplicates().values:
    if chassi in conf_dict:
        conf_data = conf_dict[chassi]
        
        # Ordenar datas e status
        dates = sorted([d for d in conf_data['data_status'] if pd.notna(d)], reverse=True)
        
        if dates:  # Se existe pelo menos uma data
            ultima_data = dates[0]
            idx_ultima = conf_data['data_status'].index(ultima_data)
            vigencia_ultimo = conf_data['vigencia'][idx_ultima]
            empresa_ultima = conf_data['empresa'][idx_ultima]
            
            # Atualizar registros correspondentes
            mask = (df_aggregate_yesterday['chassi'] == chassi)
            empresa_atual = df_aggregate_yesterday.loc[mask, 'empresa'].iloc[0]
            
            if vigencia_ultimo == 'CANCELADO':
                df_aggregate_yesterday.loc[mask, 'status'] = 'REATIVAÇÃO'
            elif empresa_ultima != empresa_atual:
                df_aggregate_yesterday.loc[mask, 'status'] = 'MIGRAÇÃO'
                df_aggregate_yesterday.loc[mask, 'migration_from'] = empresa_ultima
            else:
                df_aggregate_yesterday.loc[mask, 'status'] = 'RENOVAÇÃO'
    else:
        mask = (df_aggregate_yesterday['chassi'] == chassi)
        df_aggregate_yesterday.loc[mask, 'status'] = 'NOVO'

df = pd.concat([df_aggregate_yesterday, df_conf])
df = df.sort_values('data_status', ascending=False)
df['status'].value_counts()

status
NOVO          131
REATIVAÇÃO    104
MIGRAÇÃO       18
RENOVAÇÃO      17
Name: count, dtype: int64

verificando se existem chassis que existam nos dois

In [39]:
# Verificando chassis existentes
chassis_yesterday = set(df_aggregate_yesterday['chassi'].unique())
chassis_all_active = set(df_conf['chassi'].unique())

# Encontrando chassis que existem em ambos
chassis_comuns = chassis_yesterday.intersection(chassis_all_active)

# Encontrando chassis que só existem em df_aggregate_yesterday
chassis_apenas_yesterday = chassis_yesterday - chassis_all_active

print(f"Total de chassis únicos em df_aggregate_yesterday: {len(chassis_yesterday)}")
print(f"Total de chassis únicos em df_conf: {len(chassis_all_active)}")
print(f"Total de chassis que existem em ambos: {len(chassis_comuns)}")
print(f"Total de chassis que só existem em df_aggregate_yesterday: {len(chassis_apenas_yesterday)}")

# Mostrando exemplos de chassis que só existem em df_aggregate_yesterday
if len(chassis_apenas_yesterday) > 0:
    print("\nExemplos de chassis que só existem em df_aggregate_yesterday:")
    for chassi in list(chassis_apenas_yesterday)[:5]:  # Mostrando os 5 primeiros
        print(f"Chassi: {chassi}")
        print(df_aggregate_yesterday[df_aggregate_yesterday['chassi'] == chassi][['placa', 'chassi', 'empresa', 'status_conjunto']].to_string())
        print()

Total de chassis únicos em df_aggregate_yesterday: 252
Total de chassis únicos em df_conf: 94142
Total de chassis que existem em ambos: 135
Total de chassis que só existem em df_aggregate_yesterday: 117

Exemplos de chassis que só existem em df_aggregate_yesterday:
Chassi: 9ADG0823CDM366151
         placa             chassi   empresa status_conjunto
26360  OBF2H33  9ADG0823CDM366151  Viavante       CANCELADO

Chassi: 9BSR4X200J3916696
         placa             chassi   empresa status_conjunto
55663  FTP3532  9BSR4X200J3916696  Viavante           ATIVO

Chassi: 9532AXAZ0KE901074
         placa             chassi   empresa status_conjunto
24632  QTJ9C60  9532AXAZ0KE901074  Viavante           ATIVO
3209   QTJ9C60  9532AXAZ0KE901074    Stcoop       CANCELADO

Chassi: 9BSG6X200R4069978
         placa             chassi   empresa status_conjunto
27070  RYZ7G84  9BSG6X200R4069978  Viavante           ATIVO

Chassi: 9BSR6X200L3960958
         placa             chassi   empresa status_conjunto
